In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-270m")
# model = AutoModelForCausalLM.from_pretrained("google/gemma-3-270m")

# model = AutoModelForCausalLM.from_pretrained("allenai/OLMo-2-0425-1B")
# tokenizer = AutoTokenizer.from_pretrained("allenai/OLMo-2-0425-1B")

/Users/likhit/Documents/Coding/NLP/NLP_basics/nlp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# input = "What is Machine Learning is the best that humans have ever bulit  in there lives and tell us now check out of this really work can you  help me to start ?"
# # device = "mps" if torch.backends.mps.is_available() else "cpu"
# inputs = tokenizer(input , return_tensors="pt")
# # model
# print(inputs["input_ids"].shape)
# print(inputs["attention_mask"].shape)

In [3]:
# st_hidden_states = model.model.embed_tokens(inputs["input_ids"])
# print(st_hidden_states.shape)

In [4]:
# pos_global = model.model.
# pos_local = model.model.rotary_emb_local(st_hidden_states)

In [5]:
# model

In [6]:
# for i , layer in enumerate(model.model.embed_tokens):
#     print(i)
#     print(layer)
# device = "cpu"

In [7]:
# from transformers.models.olmo2.modeling_olmo2 import Olmo2RotaryEmbedding

# # ------------------------------------------------------------------
# # 1. Embed the prompt
# # ------------------------------------------------------------------
# input_ids = inputs["input_ids"]
# hidden = model.model.embed_tokens(input_ids)                 # (1, L, 2048)

# seq_len = input_ids.size(1)
# position_ids = torch.arange(seq_len, device=device).unsqueeze(0)   # (1, L)

# # ------------------------------------------------------------------
# # 2. Build a *causal* attention mask (exactly what HF does)
# # ------------------------------------------------------------------
# attn_mask = torch.full((seq_len, seq_len), float("-inf"), device=device)
# attn_mask = torch.triu(attn_mask, diagonal=1)               # upper triangle = -inf
# attn_mask = attn_mask.unsqueeze(0).unsqueeze(0)             # (1,1,L,L)

# # ------------------------------------------------------------------
# # 3. Pre-compute rotary cos/sin for the whole prompt
# # ------------------------------------------------------------------
# rotary_emb: Olmo2RotaryEmbedding = model.model.rotary_emb
# cos, sin = rotary_emb(position_ids)                         # (1, L, 1, head_dim)

# # ------------------------------------------------------------------
# # 4. KV-cache container (list of (k,v) per layer)
# # ------------------------------------------------------------------
# past_key_values = [None] * len(model.model.layers)

# # ------------------------------------------------------------------
# # 5. Loop over layers
# # ------------------------------------------------------------------
# for i, layer in enumerate(model.model.layers):
#     # The *exact* signature used inside the full model:
#     layer_out = layer(
#         hidden_states=hidden,
#         attention_mask=attn_mask,
#         position_ids=position_ids,
#         past_key_value=past_key_values[i],
#         use_cache=True,
#         cos=cos,                # <-- manual rotary
#         sin=sin,                # <-- manual rotary
#         output_attentions=False,
#     )

#     hidden = layer_out[0]                     # next hidden state
#     past_key_values[i] = layer_out.past_key_value

#     k, v = layer_out.past_key_value
#     print(f"[Layer {i:02d}] hidden={hidden.shape}  K={k.shape}  V={v.shape}")

In [8]:
# prompt = "What is Machine Learning is the best that humans have ever bulit  in there lives and tell us now check out of this really work can you  help me to start ?"
# device = "mps" if torch.backends.mps.is_available() else "cpu"
# inputs = tokenizer(prompt , return_tensors="pt")
# model
# print(inputs["input_ids"].shape)
# print(inputs["attention_mask"].shape)


In [9]:
model

Gemma3ForCausalLM(
  (model): Gemma3TextModel(
    (embed_tokens): Gemma3TextScaledWordEmbedding(262144, 640, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x Gemma3DecoderLayer(
        (self_attn): Gemma3Attention(
          (q_proj): Linear(in_features=640, out_features=1024, bias=False)
          (k_proj): Linear(in_features=640, out_features=256, bias=False)
          (v_proj): Linear(in_features=640, out_features=256, bias=False)
          (o_proj): Linear(in_features=1024, out_features=640, bias=False)
          (q_norm): Gemma3RMSNorm((256,), eps=1e-06)
          (k_norm): Gemma3RMSNorm((256,), eps=1e-06)
        )
        (mlp): Gemma3MLP(
          (gate_proj): Linear(in_features=640, out_features=2048, bias=False)
          (up_proj): Linear(in_features=640, out_features=2048, bias=False)
          (down_proj): Linear(in_features=2048, out_features=640, bias=False)
          (act_fn): GELUTanh()
        )
        (input_layernorm): Gemma3RMSNorm((640,), eps=1e-06)

In [10]:
# def fun(i):
#     print(f"The layer", i )
    
# for i , layers in enumerate(model.model.layers):
#     layers.self_attn.register_forward_hook(fun(i))

In [11]:
# def hook(layer_idx):
#     def f(module, input, output):
#         attn_out = output[0]
#         print(output[1])
#         print(f"[Layer {layer_idx}] attention output: {attn_out.shape}")
#     return f

# for i, layer in enumerate(model.model.layers):
#     layer.self_attn.register_forward_hook(hook(i))

# _ = model(**inputs, use_cache=True)

In [12]:
# print("Total layers hooked:", len(model.model.layers))

# fired = []
# def record_fired(layer_idx):
#     def hook(module, input, output):
#         fired.append(layer_idx)
#         print(f"[Hook fired] layer {layer_idx}")
#     return hook

# for i, layer in enumerate(model.model.layers):
#     layer.self_attn.register_forward_hook(record_fired(i))

# try:
#     _ = model(**inputs, use_cache=True)
# except Exception as e:
#     print("Error:", e)

# print("Layers that actually ran:", fired)

In [13]:
# with torch.inference_mode():
#     out = model(**inputs, use_cache=True)

# print(type(out.past_key_values))
# print(len(out.past_key_values))

# # one layer
# k0, v0 = out.past_key_values[0]
# print(k0.shape, v0.shape)

In [14]:
from transformers.cache_utils import DynamicCache
kv_cache_storage = {}
layer_computation_order = []

# Create a custom cache class that intercepts updates
class MonitoredCache(DynamicCache):
    """Custom cache that captures KV pairs as they're stored"""
    
    def update(self, key_states, value_states, layer_idx, cache_kwargs=None):
        """Called every time a layer stores its KV cache"""
        
        # Capture the cache immediately when it's stored
        kv_cache_storage[f"layer_{layer_idx}"] = {
            "key": key_states.clone().cpu(),
            "value": value_states.clone().cpu()
        }
        layer_computation_order.append(layer_idx)
        print(f"✓ Layer {layer_idx}: Captured KV during computation - Key {key_states.shape}, Value {value_states.shape}")
        
        # Call parent's update to maintain normal behavior
        return super().update(key_states, value_states, layer_idx, cache_kwargs)

# Run prefill with our custom cache
prompt = "The quick brown fox jumps over the lazy dog"
inputs = tokenizer(prompt, return_tensors="pt")

print(f"Input tokens: {inputs['input_ids'].shape[1]}\n")
print("Starting layer-by-layer computation...\n")

# Create our monitored cache
past_key_values = MonitoredCache()

with torch.no_grad():
    outputs = model(
        **inputs,
        past_key_values=past_key_values,  # Use our custom cache
        use_cache=True
    )

print(f"\n{'='*50}")
print(f"✅ Successfully captured {len(kv_cache_storage)} layers during computation!")
print(f"{'='*50}")
print(f"Computation order: {layer_computation_order}")
print()

# Verify the captured caches
for layer_name in sorted(kv_cache_storage.keys(), key=lambda x: int(x.split('_')[1])):
    cache = kv_cache_storage[layer_name]
    print(f"{layer_name}: Key {cache['key'].shape}, Value {cache['value'].shape}")

# Optional: Save to disk
# torch.save(kv_cache_storage, "olmo2_kv_cache_layer_by_layer.pt")

print(f"\n✅ Each layer's KV cache was captured DURING forward pass, not after!")

Input tokens: 10

Starting layer-by-layer computation...

✓ Layer 0: Captured KV during computation - Key torch.Size([1, 1, 10, 256]), Value torch.Size([1, 1, 10, 256])
✓ Layer 1: Captured KV during computation - Key torch.Size([1, 1, 10, 256]), Value torch.Size([1, 1, 10, 256])
✓ Layer 2: Captured KV during computation - Key torch.Size([1, 1, 10, 256]), Value torch.Size([1, 1, 10, 256])
✓ Layer 3: Captured KV during computation - Key torch.Size([1, 1, 10, 256]), Value torch.Size([1, 1, 10, 256])
✓ Layer 4: Captured KV during computation - Key torch.Size([1, 1, 10, 256]), Value torch.Size([1, 1, 10, 256])
✓ Layer 5: Captured KV during computation - Key torch.Size([1, 1, 10, 256]), Value torch.Size([1, 1, 10, 256])
✓ Layer 6: Captured KV during computation - Key torch.Size([1, 1, 10, 256]), Value torch.Size([1, 1, 10, 256])
✓ Layer 7: Captured KV during computation - Key torch.Size([1, 1, 10, 256]), Value torch.Size([1, 1, 10, 256])
✓ Layer 8: Captured KV during computation - Key torch.

In [17]:
kv_cache_storage['layer_0']

{'key': tensor([[[[ 0.0054, -0.0421,  0.0082,  ..., -2.5615,  0.7074, -3.5098],
           [-0.3547,  0.2818,  1.6888,  ..., -2.7601,  1.0438, -2.9864],
           [-2.2232,  0.7170, -0.5147,  ...,  1.3554,  0.7316, -3.4654],
           ...,
           [ 0.0380, -0.6619,  1.1123,  ..., -3.2491,  0.7625, -2.7972],
           [-1.7064,  0.2602,  0.4606,  ..., -0.2007,  1.9049, -4.6290],
           [-1.5021,  1.3075, -1.2664,  ...,  3.7362,  2.7996, -1.8984]]]]),
 'value': tensor([[[[ -0.8908,   0.2031,  -0.7874,  ...,   0.3974,   0.7160,   0.3788],
           [  9.4710,   5.7023,   3.7420,  ...,   0.5967,  -1.1231,  -1.3172],
           [ 12.3833, -20.9469,  -8.2705,  ..., -11.3474,  -4.3166,  12.1494],
           ...,
           [  8.6267,   2.7562,  -1.7541,  ...,  -1.4571,   1.5015,   1.7500],
           [ 11.9342, -14.2396,   0.2285,  ...,  -1.9927,  -0.0937,   8.7626],
           [ -8.1127,   6.9305,  -4.1905,  ..., -16.4426,  -1.0108,   1.0505]]]])}

In [15]:
# import transformers
# import inspect
# from transformers.cache_utils import DynamicCache
# print(inspect.getsource(DynamicCache.update))